In [1]:
from pyensembl import EnsemblRelease
# release 75 uses human reference genome GRCh37
ensembl_data = EnsemblRelease(75)

from joblib import Parallel, delayed

from utils import *

INFO:rpy2.situation:cffi mode is CFFI_MODE.ANY
INFO:rpy2.situation:R home found: /usr/lib/R
INFO:rpy2.situation:R library path: /usr/lib/R/lib:/usr/lib/x86_64-linux-gnu:/usr/lib/jvm/default-java/lib/server
INFO:rpy2.situation:LD_LIBRARY_PATH: /usr/lib/R/lib:/usr/lib/x86_64-linux-gnu:/usr/lib/jvm/default-java/lib/server
INFO:rpy2.rinterface_lib.embedded:Default options to initialize R: rpy2, --quiet, --no-save
INFO:rpy2.rinterface_lib.embedded:R is already initialized. No need to initialize.


---

In [8]:
%%time
vcf_read = vcf.Reader(open("test_vcf_data.txt", "r"))
out = Parallel(n_jobs=5)(delayed(process_record)(record, i) for i, record in enumerate(vcf_read))

CPU times: user 29.2 s, sys: 1.47 s, total: 30.7 s
Wall time: 1h 4min 47s


In [11]:
len(out)

11765

In [12]:
out_df = pd.concat(out)

In [16]:
# calculate VAF
out_df["VAF"] = out_df["NV"].astype(int) / out_df["NR"].astype(int)

# calculate reference reads
out_df["NWT"] = out_df["NR"].astype(int) - out_df["NV"].astype(int)

# calculate reference %
out_df["RAF"] = out_df["NWT"].astype(int) / out_df["NR"].astype(int)

TypeError: int() argument must be a string, a bytes-like object or a number, not '_collections._tuplegetter'

In [13]:
out_df

,CHROM,POS,REF,ALT,QUAL,start,end,alleles,affected_start,affected_end,...,gene_biotype5,gene_description5,gene_id6,gene_symbol6,gene_biotype6,gene_description6,gene_id7,gene_symbol7,gene_biotype7,gene_description7
0,1,1158631,A,G,2965,1158630,1158631,A;G,1158630,1158631,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1246004,A,G,2965,1246003,1246004,A;G,1246003,1246004,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1249187,G,A,2965,1249186,1249187,G;A,1249186,1249187,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1261824,G,C,2965,1261823,1261824,G;C,1261823,1261824,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1387667,C,G,2965,1387666,1387667,C;G,1387666,1387667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11760,X,154020114,C,A,2965,154020113,154020114,C;A,154020113,154020114,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11761,X,154456747,A,G,2965,154456746,154456747,A;G,154456746,154456747,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11762,X,155125435,AG,A,2973,155125434,155125436,AG;A,155125435,155125436,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11763,X,155127675,A,G,2965,155127674,155127675,A;G,155127674,155127675,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
out_df.columns

Index(['CHROM', 'POS', 'REF', 'ALT', 'QUAL', 'start', 'end', 'alleles',
       'affected_start', 'affected_end', 'BRF', 'FR', 'HP', 'HapScore', 'MGOF',
       'MMLQ', 'MQ', 'NF', 'NR', 'PP', 'QD', 'SC', 'SbPval', 'Source', 'TC',
       'TCF', 'TCR', 'TR', 'WE', 'WS', 'sample', 'GT', 'GL', 'GOF', 'GQ', 'NV',
       'ALT_type', 'gene_id', 'gene_symbol', 'gene_biotype',
       'gene_description', 'most_severe_consequence', 'gene_id1',
       'gene_symbol1', 'gene_biotype1', 'gene_description1', 'FILTER',
       'gene_id2', 'gene_symbol2', 'gene_biotype2', 'gene_description2',
       'gene_id3', 'gene_symbol3', 'gene_biotype3', 'gene_description3',
       'gene_id4', 'gene_symbol4', 'gene_biotype4', 'gene_description4',
       'gene_id5', 'gene_symbol5', 'gene_biotype5', 'gene_description5',
       'gene_id6', 'gene_symbol6', 'gene_biotype6', 'gene_description6',
       'gene_id7', 'gene_symbol7', 'gene_biotype7', 'gene_description7'],
      dtype='object')

In [21]:
out_df["ALT_type"].value_counts()

SNV    10838
MNV      891
Name: ALT_type, dtype: int64

In [22]:
out_df["most_severe_consequence"].value_counts()

intron_variant                         3942
missense_variant                       2549
synonymous_variant                     2243
non_coding_transcript_exon_variant     1184
splice_polypyrimidine_tract_variant     521
3_prime_UTR_variant                     299
splice_region_variant                   270
intergenic_variant                      262
5_prime_UTR_variant                     109
regulatory_region_variant                96
splice_donor_region_variant              85
frameshift_variant                       47
inframe_deletion                         25
inframe_insertion                        23
splice_donor_5th_base_variant            20
splice_acceptor_variant                  20
stop_gained                              17
splice_donor_variant                     14
stop_lost                                11
TF_binding_site_variant                  10
start_lost                                9
coding_sequence_variant                   7
stop_retained_variant           

In [14]:
out_df.to_csv("test_annotations.csv", index=False)

---

In [26]:
%%time

out = []
vcf_read = vcf.Reader(open("test_vcf_data.txt", "r"))

for i, record in enumerate(vcf_read):
    if i < 10:
        print(record, end = "\n\n")
        out.append(process_record(record, i))
    else:
        break

print("\nDone! Total records: {}".format(i))

Record(CHROM=1, POS=1158631, REF=A, ALT=[G])

Record(CHROM=1, POS=1246004, REF=A, ALT=[G])

Record(CHROM=1, POS=1249187, REF=G, ALT=[A])

Record(CHROM=1, POS=1261824, REF=G, ALT=[C])

Record(CHROM=1, POS=1387667, REF=C, ALT=[G])

Record(CHROM=1, POS=1585597, REF=A, ALT=[G])

Record(CHROM=1, POS=1585642, REF=G, ALT=[T])

Record(CHROM=1, POS=1586752, REF=T, ALT=[C])

Record(CHROM=1, POS=1647686, REF=A, ALT=[C])

Record(CHROM=1, POS=1647722, REF=GCTGTGACA, ALT=[TCTAGGATG])


Done! Total records: 10
CPU times: user 331 ms, sys: 32.8 ms, total: 364 ms
Wall time: 20.8 s


In [27]:
out_df = pd.concat(out)

In [31]:
# calculate VAF
out_df["VAF"] = out_df["NV"].astype(int) / out_df["NR"].astype(int)

# calculate reference reads
out_df["NWT"] = out_df["NR"].astype(int) - out_df["NV"].astype(int)

# calculate reference %
out_df["RAF"] = out_df["NWT"].astype(int) / out_df["NR"].astype(int)

In [34]:
out_df.ALT

0            G
1            G
2            A
3            C
4            G
5            G
6            T
7            C
8            C
9    TCTAGGATG
Name: ALT, dtype: object

---
# 1. Loop

In [63]:
%%time

out_df = pd.DataFrame()  # initialize empty dataframe
vcf_read = vcf.Reader(open("test_vcf_data.txt", "r"))

for i, record in enumerate(vcf_read):
    if i < 30:
        print(record, end = "\n\n")

        # unpack record into dictionary of depth 1
        record_unpacked = unpack_dict(record.__dict__)
        # unpack model._Call object
        call_dict = unpack_dict(
            extract_call(record_unpacked["samples"], format_str=record_unpacked["FORMAT"]),
        )
        # remove old keys
        del record_unpacked["samples"]
        del record_unpacked["FORMAT"]
        # merge dicts
        record_unpacked = {**record_unpacked, **call_dict}
    
        # unpack ALT
        record_unpacked = extract_alt(record_unpacked, alt_key="ALT")
        # split multiple vals
        if ";" in record_unpacked["ALT"]:
            split_vals(record_unpacked, "ALT", list_delim=";")
            split_vals(record_unpacked, "NR", list_delim=";")
            split_vals(record_unpacked, "NV", list_delim=";")
    
        # get gene where mutation is
        overlap_dict = overlap_get(record)
        try:
            # first record
            record_unpacked["gene_id"] = overlap_dict[0]["gene_id"]
            record_unpacked["gene_symbol"] = overlap_dict[0]["external_name"]
            record_unpacked["gene_biotype"] = overlap_dict[0]["biotype"]
            record_unpacked["gene_description"] = overlap_dict[0]["description"]
            if len(overlap_dict) > 1:
                for i in range(1, len(overlap_dict)):
                    record_unpacked["gene_id{}".format(i)] = overlap_dict[i]["gene_id"]
                    record_unpacked["gene_symbol{}".format(i)] = overlap_dict[i]["external_name"]
                    record_unpacked["gene_biotype{}".format(i)] = overlap_dict[i]["biotype"]
                    record_unpacked["gene_description{}".format(i)] = overlap_dict[i]["description"]
        except:
            print("No gene overlap found for {}".format(record))

        # call Ensembl VEP endpoint
        vep_dict = vep_region_post(
            record,
            hgvs=1,
            #protein=1,
            #uniprot=1,
            #GO=1,
            #vcf_string=1,
            #domains=1,
            distance=0,  # don't look for upstream/downstream effects for simplicity
        )
        # extract VEP info
        record_unpacked["most_severe_consequence"] = vep_dict["most_severe_consequence"]
        
        # get MAF and snp info if available
        if "colocated_variants" in vep_dict:
            for d in vep_dict["colocated_variants"]:
                if "minor_allele" in d:
                    record_unpacked["minor_allele"] = d["minor_allele"]
                    record_unpacked["minor_allele_freq"] = d["minor_allele_freq"]
                    record_unpacked["snp_id"] = d["id"]
        
        # add to out_df
        out_df = pd.concat([out_df, pd.DataFrame(record_unpacked, index=[i])])
    else:
        break
    
print("\nDone! Total records: {}".format(i))

Record(CHROM=1, POS=1158631, REF=A, ALT=[G])

Record(CHROM=1, POS=1246004, REF=A, ALT=[G])

Record(CHROM=1, POS=1249187, REF=G, ALT=[A])

Record(CHROM=1, POS=1261824, REF=G, ALT=[C])

Record(CHROM=1, POS=1387667, REF=C, ALT=[G])

Record(CHROM=1, POS=1585597, REF=A, ALT=[G])

Record(CHROM=1, POS=1585642, REF=G, ALT=[T])

Record(CHROM=1, POS=1586752, REF=T, ALT=[C])

Record(CHROM=1, POS=1647686, REF=A, ALT=[C])

Record(CHROM=1, POS=1647722, REF=GCTGTGACA, ALT=[TCTAGGATG])

Record(CHROM=1, POS=1647745, REF=GGCCCTTTC, ALT=[AGCCCTTTT])

Record(CHROM=1, POS=1647778, REF=C, ALT=[G])

Record(CHROM=1, POS=1647893, REF=C, ALT=[CTTTCTT])

Record(CHROM=1, POS=1647968, REF=C, ALT=[CAT])

Record(CHROM=1, POS=1647971, REF=G, ALT=[A])

Record(CHROM=1, POS=1647983, REF=TGGCTTAC, ALT=[AGGCTTAT])

Record(CHROM=1, POS=1650787, REF=TGATGCCTACATTTT, ALT=[CGATGCCTACGTTTC])

Record(CHROM=1, POS=1650807, REF=T, ALT=[C])

Record(CHROM=1, POS=1650832, REF=A, ALT=[G])

Record(CHROM=1, POS=1650845, REF=G, ALT=[A])

In [65]:
out_df

,CHROM,POS,REF,ALT,QUAL,start,end,alleles,affected_start,affected_end,...,gene_description,most_severe_consequence,minor_allele,minor_allele_freq,snp_id,gene_id1,gene_symbol1,gene_biotype1,gene_description1,FILTER
0,1,1158631,A,G,2965,1158630,1158631,A;G,1158630,1158631,...,stromal cell derived factor 4 [Source:HGNC Sym...,synonymous_variant,A,0.0467,rs6603781,NaN,NaN,NaN,NaN,NaN
1,1,1246004,A,G,2965,1246003,1246004,A;G,1246003,1246004,...,pseudouridylate synthase-like 1 [Source:HGNC S...,splice_polypyrimidine_tract_variant,A,0.0481,rs2296474,NaN,NaN,NaN,NaN,NaN
2,1,1249187,G,A,2965,1249186,1249187,G;A,1249186,1249187,...,cleavage and polyadenylation specific factor 3...,missense_variant,A,0.3101,rs12142199,NaN,NaN,NaN,NaN,NaN
3,1,1261824,G,C,2965,1261823,1261824,G;C,1261823,1261824,...,glycolipid transfer protein domain containing ...,intron_variant,G,0.1745,rs307348,NaN,NaN,NaN,NaN,NaN
4,1,1387667,C,G,2965,1387666,1387667,C;G,1387666,1387667,...,"ATPase family, AAA domain containing 3C [Sourc...",intron_variant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,1585597,A,G,2962,1585596,1585597,A;G,1585596,1585597,...,cyclin-dependent kinase 11B [Source:HGNC Symbo...,intron_variant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,1585642,G,T,2956,1585641,1585642,G;T,1585641,1585642,...,cyclin-dependent kinase 11B [Source:HGNC Symbo...,intron_variant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1,1586752,T,C,2965,1586751,1586752,T;C,1586751,1586752,...,cyclin-dependent kinase 11B [Source:HGNC Symbo...,intron_variant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1647686,A,C,2962,1647685,1647686,A;C,1647685,1647686,...,cyclin-dependent kinase 11A [Source:HGNC Symbo...,intron_variant,A,0.2480,rs909823,ENSG00000268575,RP1-283E3.8,processed_transcript,None,NaN
1,1,1647722,GCTGTGACA,TCTAGGATG,2914,1647721,1647730,GCTGTGACA;TCTAGGATG,1647722,1647730,...,cyclin-dependent kinase 11A [Source:HGNC Symbo...,intron_variant,NaN,NaN,NaN,ENSG00000268575,RP1-283E3.8,processed_transcript,None,alleleBias


In [52]:
overlap_dict

[{'seq_region_name': '1',
  'start': 1634169,
  'strand': -1,
  'canonical_transcript': 'ENST00000404249.3',
  'id': 'ENSG00000008128',
  'version': 18,
  'description': 'cyclin-dependent kinase 11A [Source:HGNC Symbol;Acc:1730]',
  'external_name': 'CDK11A',
  'end': 1655766,
  'assembly_name': 'GRCh37',
  'logic_name': 'ensembl_havana_gene_homo_sapiens_37',
  'biotype': 'protein_coding',
  'feature_type': 'gene',
  'source': 'ensembl_havana',
  'gene_id': 'ENSG00000008128'},
 {'strand': -1,
  'start': 1634175,
  'seq_region_name': '1',
  'canonical_transcript': 'ENST00000598846.1',
  'description': None,
  'id': 'ENSG00000268575',
  'version': 1,
  'end': 1669127,
  'external_name': 'RP1-283E3.8',
  'biotype': 'processed_transcript',
  'logic_name': 'havana_homo_sapiens_37',
  'assembly_name': 'GRCh37',
  'source': 'havana',
  'feature_type': 'gene',
  'gene_id': 'ENSG00000268575'}]

In [61]:
vep_dict["most_severe_consequence"]

'missense_variant'

In [53]:
[vep_dict["transcript_consequences"][x]["gene_symbol"] for x in range(len(vep_dict["transcript_consequences"]))]

['CDK11A',
 'CDK11A',
 'CDK11A',
 'CDK11A',
 'CDK11A',
 'CDK11A',
 'CDK11A',
 'CDK11A',
 'CDK11A',
 'CDK11A',
 'CDK11A',
 'CDK11A',
 'RP1-283E3.8']

In [54]:
[vep_dict["transcript_consequences"][x]["biotype"] for x in range(len(vep_dict["transcript_consequences"]))]

['protein_coding',
 'retained_intron',
 'protein_coding',
 'protein_coding',
 'protein_coding',
 'protein_coding',
 'protein_coding',
 'protein_coding',
 'nonsense_mediated_decay',
 'protein_coding',
 'retained_intron',
 'nonsense_mediated_decay',
 'processed_transcript']

In [55]:
[vep_dict["transcript_consequences"][x]["consequence_terms"] for x in range(len(vep_dict["transcript_consequences"]))]

[['missense_variant'],
 ['non_coding_transcript_exon_variant'],
 ['missense_variant'],
 ['missense_variant'],
 ['missense_variant'],
 ['missense_variant'],
 ['missense_variant'],
 ['missense_variant'],
 ['missense_variant', 'NMD_transcript_variant'],
 ['missense_variant'],
 ['non_coding_transcript_exon_variant'],
 ['missense_variant', 'NMD_transcript_variant'],
 ['non_coding_transcript_exon_variant']]

In [62]:
vep_dict["transcript_consequences"]

[{'gene_symbol': 'CDK11A',
  'hgnc_id': 1730,
  'uniparc': ['UPI0000470903'],
  'hgvsp': 'ENSP00000348529.2:p.Arg59Trp',
  'polyphen_score': 0.991,
  'hgvsc': 'ENST00000356200.3:c.175C>T',
  'codons': 'Cgg/Tgg',
  'cdna_start': 410,
  'strand': -1,
  'sift_prediction': 'deleterious_low_confidence',
  'cds_end': 175,
  'gene_id': 'ENSG00000008128',
  'protein_start': 59,
  'go': 'GO:0004672:protein_kinase_activity,GO:0004674:protein_serine/threonine_kinase_activity,GO:0004713:protein_tyrosine_kinase_activity,GO:0005524:ATP_binding,GO:0005634:nucleus,GO:0005737:cytoplasm,GO:0006468:protein_phosphorylation,GO:0016772:transferase_activity_-_transferring_phosphorus-containing_groups',
  'cds_start': 175,
  'impact': 'MODERATE',
  'cdna_end': 410,
  'sift_score': 0.01,
  'variant_allele': 'A',
  'trembl': ['Q5QPR4_HUMAN', 'Q4VBY6_HUMAN'],
  'protein_end': 59,
  'gene_symbol_source': 'HGNC',
  'transcript_id': 'ENST00000356200',
  'polyphen_prediction': 'probably_damaging',
  'amino_acids': '

In [25]:
# calculate VAF
out_df["VAF"] = out_df["NV"].astype(int) / out_df["NR"].astype(int)

# calculate reference reads
out_df["NWT"] = out_df["NR"].astype(int) - out_df["NV"].astype(int)

In [22]:
out_df.columns

Index(['CHROM', 'POS', 'REF', 'ALT', 'QUAL', 'start', 'end', 'alleles',
       'affected_start', 'affected_end', 'BRF', 'FR', 'HP', 'HapScore', 'MGOF',
       'MMLQ', 'MQ', 'NF', 'NR', 'PP', 'QD', 'SC', 'SbPval', 'Source', 'TC',
       'TCF', 'TCR', 'TR', 'WE', 'WS', 'sample', 'GT', 'GL', 'GOF', 'GQ', 'NV',
       'FILTER', 'NR1', 'NV1'],
      dtype='object')

In [29]:
out_df["TC"]

0        160
1        152
2        137
3        136
4        137
        ... 
11760    217
11761    226
11762    208
11763    134
11764    232
Name: TC, Length: 11765, dtype: int64

In [31]:
out_df[["TC","NR","NV","NWT","VAF"]]

,TC,NR,NV,NWT,VAF
0,160,160,156,4,0.975000
1,152,152,148,4,0.973684
2,137,137,135,2,0.985401
3,136,136,134,2,0.985294
4,137,137,133,4,0.970803
...,...,...,...,...,...
11760,217,217,216,1,0.995392
11761,226,226,224,2,0.991150
11762,208,208,178,30,0.855769
11763,134,134,134,0,1.000000


In [33]:
out_df.loc[out_df.TC!=out_df.NR,:]

,CHROM,POS,REF,ALT,QUAL,start,end,alleles,affected_start,affected_end,...,GT,GL,GOF,GQ,NV,FILTER,NR1,NV1,VAF,NWT
39,1,6219287,TCACACA,TCACA;T,2997,6219286,6219293,TCACACA;TCACA;T,6219287,6219293,...,1/2,-1.0;-1.0;-1.0,6.0,99,108,NaN,172,108,0.627907,64
414,1,91859795,TATGTGA,CATGTGA;CATGTGG,2962,91859794,91859801,TATGTGA;CATGTGA;CATGTGG,91859795,91859801,...,1/2,-1.0;-1.0;-1.0,1.0,99,208,NaN,209,95,0.995215,1
713,1,154516571,TATGCACG,CATGCACA;CATGCACG,2944,154516570,154516578,TATGCACG;CATGCACA;CATGCACG,154516571,154516578,...,1/2,-1.0;-1.0;-1.0,4.0,99,104,NaN,199,191,0.522613,95
771,1,156914240,CCATT,C;CCATTCATT,2991,156914239,156914244,CCATT;C;CCATTCATT,156914240,156914244,...,1/2,-1.0;-1.0;-1.0,3.0,99,66,NaN,128,66,0.515625,62
1008,1,182429295,CTGTG,C;CTG,2995,182429294,182429299,CTGTG;C;CTG,182429295,182429299,...,1/2,-1.0;-1.0;-1.0,4.0,99,103,NaN,201,103,0.512438,98
1348,1,235543476,TTGTGTGTGTGTGTGTGTGTGTG,T;TTGTGTGTG,2997,235543475,235543498,TTGTGTGTGTGTGTGTGTGTGTG;T;TTGTGTGTG,235543476,235543498,...,1/2,-1.0;-1.0;-1.0,3.0,99,17,alleleBias,104,17,0.163462,87
2626,3,64527465,C,A;T,2906,64527464,64527465,C;A;T,64527464,64527465,...,1/2,-1.0;-1.0;-1.0,8.0,99,75,NaN,169,92,0.443787,94
2779,3,113376110,TTGC,T;TTGCTGCTGC,2994,113376109,113376113,TTGC;T;TTGCTGCTGC,113376110,113376113,...,2/1,-1.0;-1.0;-1.0,5.0,99,42,NaN,151,42,0.278146,109
3386,4,87024228,C,CTGTGTA;CTGTGTGTGTG,2997,87024227,87024228,C;CTGTGTA;CTGTGTGTGTG,87024228,87024228,...,1/2,-1.0;-1.0;-1.0,8.0,99,62,NaN,141,59,0.416107,87
3577,4,153884092,GGTGTGTGT,G;GGTGTGTGTGT,2997,153884091,153884100,GGTGTGTGT;G;GGTGTGTGTGT,153884092,153884100,...,1/2,-1.0;-1.0;-1.0,8.0,99,42,NaN,113,42,0.371681,71
